In [39]:
import re

_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]

In [42]:
import os
import numpy as np
from natsort import natsorted, ns

def create_img_list(root_path,images,patch_path):
    filename2value = lambda x: x[:x.rfind('.')].split('_')[-1]
    # ATTENTION: modify when path changes
    #PATCH_PATH = "epi/4/patches_9*33"
    
    #listdir已经是乱序
    img_files = []
    
    for img in images:
        pt_folder = root_path + "/" + img + "/" + patch_path
        dirs = os.listdir(pt_folder)
        dirs.sort(key=natural_sort_key)
        #dirs.sort(key = str.lower) 
        patches = []
        with open(root_path + "/" + img + "/" + 'full_test.txt', 'w') as full_test_txt:
            for dir in dirs:
                files = os.listdir("%s/%s"%(pt_folder,dir))
                files.sort(key=natural_sort_key) #sort(key = str.lower)
                #sorted(files)
                for f in files:
                    #list的合并
                    line = "%s/%s/%s/%s"%(img,patch_path,dir,f)
                    full_test_txt.write(line+"\n") #write() wothout \n
                    img_files.append(line)
        full_test_txt.close()

    #print len(img_files),img_files[0]

    ##################### 再次乱序，特别注意，之后分别生成lmdb、hdf5时不要再打乱顺序 #################################
    np.random.shuffle(img_files) 

    #就因为对其到了for身上，出现了异常
    #200000 batch_size=500
    offset = 2
    accuracy = 0.07
    train_num = 220000 * len(sub_folders)
    with open('train.txt', 'w') as train_txt:
        for f in img_files[:train_num]:
            label = int((float(filename2value(f)) + offset)/accuracy + 0.5) #disp四舍五入
            img_label_line = '{} {}\n'.format(f,label)
            train_txt.write(img_label_line)
    print str(train_num) + " train images..."

    #20000 batch_size=500
    val_num = 20000 * len(sub_folders)
    with open('val.txt', 'w') as val_txt:
        for f in img_files[train_num:train_num + val_num]:
            label = int((float(filename2value(f)) + offset)/accuracy + 0.5) #disp四舍五入
            img_label_line = '{} {}\n'.format(f,label)
            val_txt.write(img_label_line)
    print str(val_num) + " validate images..."

    #left...
    with open('test.txt', 'w') as test_txt:
        for f in img_files[train_num + val_num:]:
            img_line = '{}\n'.format(f)
            test_txt.write(img_line)
    print str(len(img_files[train_num + val_num:])) + " test images..."

In [43]:
root_path = "/home/hzdz505/dl/datasets/full_data-depth/training"

#img_folder = "/home/hzdz505/dl/datasets/full_data-depth/training/sideboard/epi/4/patches_9*33"
imgs = ["sideboard","cotton","boxes","dino"]
patch_path = "epi/4/patches_9*33"
create_img_list(root_path,imgs,patch_path)

880000 train images...
80000 validate images...
88576 test images...
